# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import statsmodels.api as sm
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
# How many viewers also clicked?
viewers = df[df['action'] == 'view']
clickers = viewers[viewers['id'].isin(df[df['action'] == 'click']['id'])]
viewer_clickers_count = len(clickers)
viewer_clickers_count

1860

In [3]:
#Are there any anomalies with the data; did anyone click who didn't view? 
anomalies = clickers[~clickers['id'].isin(df[df['action'] == 'view']['id'])],
print('Number of clickers who did not view:', anomalies)
   

Number of clickers who did not view: (Empty DataFrame
Columns: [timestamp, id, group, action]
Index: [],)


In [4]:
# Is there any overlap between the control and experiment groups?
control_ids = set(df[df['group'] == 'control']['id'])
experiment_ids = set(df[df['group'] == 'experiment']['id'])
group_overlap = control_ids.intersection(experiment_ids)
print('Number of control and experimant overlap:', group_overlap)

Number of control and experimant overlap: set()


In [5]:
df['id'].value_counts()


363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
print('''
Null Hypothesis: The proportion of users who clicked in the experimental group is equal t
o or less than the proportion in the control group.
Alternative Hypothesis: The proportion of users who clicked in the experimental group is
significantly greater than the proportion in the control group.
''')
experi_homepage_clicks = len(df[(df['group'] == 'experiment') & (df['action'] == 'click'
)])
control_group_clicks = len(df[(df['group'] == 'control') & (df['action'] == 'click')])
experi_homepage_total = len(df[df['group'] == 'experiment'])
control_group_total = len(df[df['group'] == 'control'])
# Perform the Z-test for proportions
z_score, p_value = sm.stats.proportions_ztest([experi_homepage_clicks, control_group_clicks], [experi_homepage_total, control_group_total], alternative='larger')
print('P_value =',p_value)


Null Hypothesis: The proportion of users who clicked in the experimental group is equal t
o or less than the proportion in the control group.
Alternative Hypothesis: The proportion of users who clicked in the experimental group is
significantly greater than the proportion in the control group.

P_value = 0.02660147426144761


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
#Your code here
control_group_ctr = control_group_clicks / control_group_total

# Estimate the expected number of clicks for the experimental group
expected_clicks_experi_group = experi_homepage_total * control_group_ctr
expected_clicks_experi_group

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [8]:
#Your code here
from scipy import stats
std = np.sqrt(experi_homepage_total * control_group_ctr * (1 - control_group_ctr))
std


25.888551746650283

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
#Your code here
z_score = (experi_homepage_clicks - expected_clicks_experi_group) / std
p_value = 1 - stats.norm.cdf((z_score))
z_score, p_value


(2.71607302278784, 0.0033030672759265522)

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The result matches the previous statistal test as they both provide strong basis to
reject the null hypothesis. They both suggest that the experimental page design is more effective
than the control page design. However, the p-vale is slightly lower

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.